In [25]:
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeRevol,BRepPrimAPI_MakeCylinder
from OCC.Core.gp import gp_Pnt, gp_Ax1, gp_Dir,gp_Trsf,gp_Vec,gp_Ax2
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge,BRepBuilderAPI_MakeWire,BRepBuilderAPI_Transform
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Cut,BRepAlgoAPI_Fuse
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
import math

def create_coffee_mug():
    """创建一个带把手的咖啡杯"""
    
    print("☕ 创建咖啡杯模型")
    print("=" * 40)
    
    # 1. 创建杯身（圆柱体）
    cup_body = BRepPrimAPI_MakeCylinder(25, 40).Shape()
    print("✅ 杯身创建完成: 半径25, 高度40")
    
    # 2. 创建内部空腔（稍小的圆柱体）
    inner_cavity = BRepPrimAPI_MakeCylinder(22, 38).Shape()
    
    # 移动内部空腔向上2单位（留底部厚度）
    transform = gp_Trsf()
    transform.SetTranslation(gp_Vec(0, 0, 2))
    moved_cavity = BRepBuilderAPI_Transform(inner_cavity, transform).Shape()
    print("✅ 内部空腔创建并定位完成")
    
    # 3. 挖空杯身（杯身 - 内部空腔）
    hollow_operation = BRepAlgoAPI_Cut(cup_body, moved_cavity)
    hollow_cup = hollow_operation.Shape()
    print("✅ 杯身挖空完成")
    
    # 4. 创建把手（环形拉伸）
    # 创建把手的截面轮廓（圆形）
    handle_center = gp_Pnt(35, 0, 20)  # 把手中心位置
    handle_profile = BRepPrimAPI_MakeCylinder(3, 5).Shape()  # 截面圆
    
    # 把手路径（半圆弧）
    import math
    handle_points = []
    for angle in range(0, 181, 20):  # 0到180度，步长20度
        rad = math.radians(angle)
        x = 35 + 8 * math.cos(rad)
        y = 8 * math.sin(rad)
        z = 20
        handle_points.append(gp_Pnt(x, y, z))
    
    # 创建把手路径
    handle_edges = []
    for i in range(len(handle_points)-1):
        edge = BRepBuilderAPI_MakeEdge(handle_points[i], handle_points[i+1]).Edge()
        handle_edges.append(edge)
    
    handle_wire_maker = BRepBuilderAPI_MakeWire()
    for edge in handle_edges:
        handle_wire_maker.Add(edge)
    handle_path = handle_wire_maker.Wire()
    
    print("✅ 把手路径创建完成")
    
    # 5. 创建把手（圆形截面沿路径扫掠）
    # 简化版：使用环形近似把手
    handle_torus = BRepPrimAPI_MakeCylinder(
        gp_Ax2(gp_Pnt(30, 0, 20), gp_Dir(0, 1, 0)), 10, 5
    ).Shape()
    
    # 创建把手孔
    handle_hole = BRepPrimAPI_MakeCylinder(
        gp_Ax2(gp_Pnt(30, -1, 20), gp_Dir(0, 1, 0)), 8, 17
    ).Shape()
    
    handle_solid = BRepAlgoAPI_Cut(handle_torus, handle_hole).Shape()
    print("✅ 把手创建完成")
    
    # 6. 组合杯身和把手
    final_mug = BRepAlgoAPI_Fuse(hollow_cup, handle_solid).Shape()
    print("✅ 咖啡杯组装完成")
    
    return final_mug

# 执行建模
coffee_mug = create_coffee_mug()

if coffee_mug:
    # 分析模型
    def analyze_shape_advanced(shape):
        """分析形状的高级属性"""
        from OCC.Core.GProp import GProp_GProps
        from OCC.Core.BRepGProp import brepgprop_VolumeProperties, brepgprop_SurfaceProperties
        
        # 体积和表面积
        props = GProp_GProps()
        brepgprop_VolumeProperties(shape, props)
        volume = props.Mass()
        
        surf_props = GProp_GProps()
        brepgprop_SurfaceProperties(shape, surf_props)
        surface_area = surf_props.Mass()
        
        print(f"📊 模型分析:")
        print(f"   体积: {volume:.2f} 立方单位")
        print(f"   表面积: {surface_area:.2f} 平方单位")
        print(f"   等效球半径: {(3*volume/(4*math.pi))**(1/3):.2f}")
    
    analyze_shape_advanced(coffee_mug)
    
    # 可视化
    renderer = JupyterRenderer()
    renderer.DisplayShape(coffee_mug, shape_color="#8E44AD")
    renderer.Display()

☕ 创建咖啡杯模型
✅ 杯身创建完成: 半径25, 高度40
✅ 内部空腔创建并定位完成
✅ 杯身挖空完成
✅ 把手路径创建完成
✅ 把手创建完成
✅ 咖啡杯组装完成
📊 模型分析:
   体积: 21223.39 立方单位
   表面积: 16019.04 平方单位
   等效球半径: 17.18


C:\Users\asd56\AppData\Local\Temp\ipykernel_36304\3530067798.py:92: DeprecationWarning: Call to deprecated function brepgprop_VolumeProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.VolumeProperties
  brepgprop_VolumeProperties(shape, props)
C:\Users\asd56\AppData\Local\Temp\ipykernel_36304\3530067798.py:96: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(shape, surf_props)


NameError: name 'handle_center' is not defined